# Regression Analysis and Resampling Methods

### Authors:
1. Britt
2. Pragay Shourya Moudgil

## Imports

In [1]:
# Imports:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn import linear_model
from sklearn.utils import resample
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import mean_squared_error
from imageio import imread
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler




In [2]:
class Metrics():

    def __init__(self, y_target, y_pred):
        
        self.y_target = y_target
        self.y_pred = y_pred


    def r2_score(self):
        a = self.y_pred - np.mean(self.y_pred)
        b = self.y_target - np.mean(self.y_target)
        return (np.sum(a * b)) ** 2 / (np.sum(a ** 2) * np.sum(b ** 2))

            
    
    def mean_squared_error(self):
        return (np.sum(np.square(self.y_target - self.y_pred ))/len(self.y_target))
    

In [5]:
class DataGenerator():

    def __init__(self, x, y):
        self.x, self.y,  = x, y 
        self.frankf = self.FrankeFunction()


    def FrankeFunction(self):

        term1 = 0.75*np.exp(-(0.25*(9*self.x-2)**2) - 0.25*((9*self.y-2)**2))
        term2 = 0.75*np.exp(-((9*self.x+1)**2)/49.0 - 0.1*(9*self.y+1))
        term3 = 0.5*np.exp(-(9*self.x-7)**2/4.0 - 0.25*((9*self.y-3)**2))
        term4 = -0.2*np.exp(-(9*self.x-4)**2 - (9*self.y-7)**2)

        self.frankf = term1 + term2 + term3 + term4
        
        return self.frankf
    
    def plot_franke(self, z = None, name_fig = None):
        
        z = self.frankf

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Plot the surface.
        surf = ax.plot_surface(self.x, self.y, z, cmap=cm.coolwarm,
                            linewidth=0, antialiased=False)

        # Customize the z axis.
        ax.set_zlim(-0.10, 1.40)
        ax.zaxis.set_major_locator(LinearLocator(10))
        ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

        # Add a color bar which maps values to colors.
        fig.colorbar(surf, shrink=0.5, aspect=5)
        plt.savefig(name_fig, dpi = 600, bbox_inches='tight')
        plt.show()

In [6]:
# Utils:
def determine_X(poly_deg, x, y, intercept = False):

    X = np.ones((len(x),int((poly_deg+1)*(poly_deg+2)/2)))
    for i in range(1,poly_deg+1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = (x**(i-k))*(y**k)

    if intercept is False:
        X = X[:,1:]

    
    return X

def scaling_mean(X_train, X_test):
    mean = np.mean(X_train, axis = 0)
    return X_train - mean, X_test - mean

def prep_train_test(X, z, train_size = 0.66):

    return train_test_split(X, z, train_size=train_size)

def bootstrap(x,
              y,
              z,
              poly_deg,
              model
              ):
    #to do

    pass

def evaluate_stats(y_pred, y_input):
    return [np.mean( np.mean((y_input[:, np.newaxis] - y_pred)**2, axis=1, keepdims=True)),
            np.mean( (y_input - np.mean(y_pred, axis=1, keepdims=True))**2),
            np.mean( np.var(y_pred, axis=1, keepdims=True))
            ] 


In [7]:
class PlotResults():

    def __init__(self):
        pass

    def plot_polynomial(self, 
                    x_axis_val,
                    trainvalues,
                    testvalues = None,
                    title = None,
                    y_label = None,
                    x_label = None,
                    train_label = None,
                    test_label = None,
                    ):
        # Plot data for MSE
        plt.plot(x_axis_val, trainvalues, label=train_label)

        if test_label is not None:
            plt.plot(x_axis_val, testvalues, label=test_label)

        plt.title(title)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.xticks(np.arange(x_axis_val[0], x_axis_val[-1]+1, step=1))
        plt.legend()
        plt.show()

In [8]:
class OLS():

    def __init__(self, x, y, z, poly_deg, sk_learn = False):
        self.x, self.y, self.z, self.poly_deg, self.sk_learn = np.ravel(x), np.ravel(y), np.ravel(z), poly_deg, sk_learn

    def _beta(self, X, z):
        #calculate beta
        return (np.linalg.inv(X.T @ X) @ X.T ) @ z
    
    def predict(self, X_input):
        return X_input @ self.betahat_train
    
    def fit(self, train_size = 0.66):
        
        X = determine_X(poly_deg=self.poly_deg, x = self.x, y= self.y, intercept = False)

        self.X_train, self.X_test, self.z_train, self.z_test = prep_train_test(X=X, z=self.z, train_size=train_size)

        self.X_train, self.X_test = scaling_mean(self.X_train, self.X_test)
        
        if self.sk_learn is False:
            self.betahat_train = self._beta(X=self.X_train, z=self.z_train)

            return self.betahat_train
        else:
            self.model = linear_model.LinearRegression(fit_intercept=False).fit(self.X_train, self.z_train)
            return self.model
    
    def evaluate(self):

        if self.sk_learn is False:
            ztilde_train = self.predict(self.X_train)
            ztilde_test = self.predict(self.X_test)
        
        else:
            ztilde_train = self.model.predict(self.X_train)
            ztilde_test = self.model.predict(self.X_test)


        return [Metrics(ztilde_train, self.z_train).mean_squared_error(),
                Metrics(ztilde_test, self.z_test).mean_squared_error(),
                Metrics(ztilde_train, self.z_train).r2_score(),
                Metrics(ztilde_test, self.z_test).r2_score()]

     

In [9]:
class RidgeRegression():

    def __init__(self, x, y, z, poly_deg, ld):

        self.x, self.y, self.z, self.poly_deg, self.ld = np.ravel(x), np.ravel(y), np.ravel(z), poly_deg, ld

    def _beta(self, X, z, ld):
        #calculate beta for ridge
        xtxinv_r = np.linalg.inv((X.T.dot(X) + ld*np.identity(X.shape[1])))
        return xtxinv_r.dot(X.T).dot(z)

    def predict(self, X_input):
        return X_input @ self.betahat_train
    

    def fit(self, train_size = 0.66):
            
        X = determine_X(poly_deg=self.poly_deg, x = self.x, y= self.y, intercept = False)

        self.X_train, self.X_test, self.z_train, self.z_test = prep_train_test(X=X, z=self.z, train_size=train_size)

        self.X_train, self.X_test = scaling_mean(self.X_train, self.X_test)
        
        self.betahat_train = self._beta(X=self.X_train, z=self.z_train, ld=self.ld)

        return self.betahat_train
    
    def evaluate(self):

        ztilde_train = self.predict(self.X_train)
        ztilde_test = self.predict(self.X_test)

        return [Metrics(ztilde_train, self.z_train).mean_squared_error(),
                Metrics(ztilde_test, self.z_test).mean_squared_error(),
                Metrics(ztilde_train, self.z_train).r2_score(),
                Metrics(ztilde_test, self.z_test).r2_score()]

In [10]:
class LassoRegression():

    def __init__(self, x, y, z, poly_deg, ld):

        self.x, self.y, self.z, self.poly_deg, self.ld = np.ravel(x), np.ravel(y), np.ravel(z), poly_deg, ld

    def fit(self, train_size = 0.66):
            
        X = determine_X(poly_deg=self.poly_deg, x = self.x, y= self.y, intercept = False)

        
        self.X_train, self.X_test, self.z_train, self.z_test = prep_train_test(X=X, z=self.z, train_size=train_size)

        self.model = linear_model.Lasso(alpha=self.ld, fit_intercept=False)

        self.X_train, self.X_test = scaling_mean(self.X_train, self.X_test)
        
        self.model.fit(self.X_train, self.z_train)

        print(self.model.coef_)

        return self.model.coef_
    
    def evaluate(self):

        ztilde_train = self.model.predict(self.X_train)
        ztilde_test = self.model.predict(self.X_test)

        return [Metrics(ztilde_train, self.z_train).mean_squared_error(),
                Metrics(ztilde_test, self.z_test).mean_squared_error(),
                Metrics(ztilde_train, self.z_train).r2_score(),
                Metrics(ztilde_test, self.z_test).r2_score()]

In [11]:
# Make data.
x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)
x,y = np.meshgrid(x, y)
z = DataGenerator(x,y).FrankeFunction()

In [ ]:
temp = DataGenerator(x, y)
temp.plot_franke(name_fig='Frankie Function.png')

In [ ]:
polynomialvalues = np.arange(start=1, stop=6, step=1)
OLS_MSE_trainvalues = np.zeros(len(polynomialvalues))
OLS_MSE_testvalues = np.zeros(len(polynomialvalues))
OLS_R2_trainvalues = np.zeros(len(polynomialvalues))
OLS_R2_testvalues = np.zeros(len(polynomialvalues))
Betavalues = np.zeros([len(polynomialvalues), int((polynomialvalues[-1] + 1) * (polynomialvalues[-1] + 2) / 2)])

# Loop through polynomial degrees and evaluate the model
for i, pvalue in enumerate(polynomialvalues):
    ols_regressor = OLS(x, y, z, poly_deg=pvalue, sk_learn=False)
    Betavalues[i, :int((i + 2) * (i + 3) / 2) - 1] = ols_regressor.fit()
    OLS_MSE_trainvalues[i], OLS_MSE_testvalues[i], OLS_R2_trainvalues[i], \
        OLS_R2_testvalues[i] = ols_regressor.evaluate()

beta_indexes = np.arange(start=0,stop=int((polynomialvalues[-1]+1)*(polynomialvalues[-1]+2)/2),step=1)
Betavalues[Betavalues==0] = np.nan 
PlotResults().plot_polynomial(x_axis_val=polynomialvalues,
                              trainvalues=OLS_MSE_trainvalues,
                              testvalues=OLS_MSE_testvalues,
                              title='Mean Square Error for OLS',
                              y_label= 'MSE',
                              x_label = 'Polynomial degree',
                              train_label='Training',
                              test_label='Testing',)

PlotResults().plot_polynomial(x_axis_val=polynomialvalues,
                              trainvalues=OLS_R2_trainvalues,
                              testvalues=OLS_R2_testvalues,
                              title='R2 for OLS',
                              y_label= 'R2',
                              x_label = 'Polynomial degree',
                              train_label='Training',
                              test_label='Testing')
PlotResults().plot_polynomial(x_axis_val=beta_indexes,
                              trainvalues=Betavalues.T,
                              testvalues=None,
                              title='Beta values for OLS',
                              x_label='Beta Index',
                              train_label=['p = 1',
                                           'p = 2',
                                           'p = 3',
                                           'p = 4',
                                           'p = 5'],
                            )

In [ ]:
labdavalues = [10**a for a in range(-5, 1)]
polynomialvalues = [5]
ridge_MSE_trainvalues = np.zeros(len(labdavalues))
ridge_MSE_testvalues = np.zeros(len(labdavalues))
ridge_R2_trainvalues = np.zeros(len(labdavalues))
ridge_R2_testvalues = np.zeros(len(labdavalues))
Betavalues = np.zeros([len(labdavalues), int((polynomialvalues[-1] + 1) * (polynomialvalues[-1] + 2) / 2)-1])

# Loop through polynomial degrees and evaluate the model
for i, ld in enumerate(labdavalues):
    ridge_regressor = RidgeRegression(x, y, z, poly_deg=polynomialvalues[-1], ld=ld)
    Betavalues[i] = ridge_regressor.fit()
    ridge_MSE_trainvalues[i], ridge_MSE_testvalues[i], ridge_R2_trainvalues[i], \
        ridge_R2_testvalues[i] = ridge_regressor.evaluate()
    
beta_indexes = np.arange(start=0,stop=int((polynomialvalues[-1]+1)*(polynomialvalues[-1]+2)/2 - 1),step=1)
Betavalues[Betavalues==0] = np.nan 
PlotResults().plot_polynomial(x_axis_val= np.log10(labdavalues),
                              trainvalues=ridge_MSE_trainvalues,
                              testvalues=ridge_MSE_testvalues,
                              title='Mean Square Error for Ridge with p=5',
                              y_label= 'MSE',
                              x_label = 'log10(Lambda Values)',
                              train_label='Training',
                              test_label='Testing')

PlotResults().plot_polynomial(x_axis_val= np.log10(labdavalues),
                              trainvalues=ridge_R2_trainvalues,
                              testvalues=ridge_R2_testvalues,
                              title='R2 for Ridge with p=5',
                              y_label= 'R2',
                              x_label = 'log10(Lambda Values)',
                              train_label='Training',
                              test_label='Testing')
PlotResults().plot_polynomial(x_axis_val=beta_indexes,
                              trainvalues=Betavalues.T,
                              testvalues=None,
                              title='Beta values for Ridge with p=5',
                              x_label='Beta Index',
                              train_label=labdavalues,
                            )


In [ ]:
labdavalues = [10**a for a in range(-5, 1)]
polynomialvalues = [5]
lasso_MSE_trainvalues = np.zeros(len(labdavalues))
lasso_MSE_testvalues = np.zeros(len(labdavalues))
lasso_R2_trainvalues = np.zeros(len(labdavalues))
lasso_R2_testvalues = np.zeros(len(labdavalues))


# Loop through polynomial degrees and evaluate the model
for i, ld in enumerate(labdavalues):
    lasso_regressor = LassoRegression(x, y, z, poly_deg=polynomialvalues[-1], ld=ld)
    Betavalues[i] = lasso_regressor.fit()
    lasso_MSE_trainvalues[i], lasso_MSE_testvalues[i], lasso_R2_trainvalues[i], \
        lasso_R2_testvalues[i] = lasso_regressor.evaluate()

beta_indexes = np.arange(start=0,stop=int((polynomialvalues[-1]+1)*(polynomialvalues[-1]+2)/2 - 1),step=1)
Betavalues[Betavalues==0] = np.nan 

PlotResults().plot_polynomial(x_axis_val= np.log10(labdavalues),
                              trainvalues=lasso_MSE_trainvalues,
                              testvalues=lasso_MSE_testvalues,
                              title='Mean Square Error for Lasso with p=5',
                              y_label= 'MSE',
                              x_label = 'log10(Lambda values)',
                              train_label='Training',
                              test_label='Testing')

PlotResults().plot_polynomial(x_axis_val= np.log10(labdavalues),
                              trainvalues=lasso_R2_trainvalues,
                              testvalues=lasso_R2_testvalues,
                              title='R2 for Lasso with p=5',
                              y_label= 'R2',
                              x_label = 'log10(Lambda Values)',
                              train_label='Training',
                              test_label='Testing')
PlotResults().plot_polynomial(x_axis_val=beta_indexes,
                              trainvalues=Betavalues.T,
                              testvalues=None,
                              title='Beta values for Lasso with p=5',
                              x_label='Beta Index',
                              train_label=labdavalues,
                            )


## Part f

In [ ]:
np.random.seed(2018)

n = 40
n_boostraps = 100
maxdegree = 14
polynomialvalues = np.arange(start=1, stop=maxdegree+1, step=1)

error = np.zeros(maxdegree)
bias = np.zeros(maxdegree)
variance = np.zeros(maxdegree)


for i, deg in tqdm(enumerate(polynomialvalues), ncols = 50, total=len(polynomialvalues)):
    X = determine_X(poly_deg=deg, x = np.ravel(x), y = np.ravel(y), intercept=False)
    X_train, X_test, z_train, z_test = prep_train_test(X=X, z=np.ravel(z), train_size=0.66)

    X_train, X_test = scaling_mean(X_train, X_test)
    model = linear_model.LinearRegression(fit_intercept=False)

    z_pred = np.empty((z_test.shape[0], n_boostraps))
    for j in range(n_boostraps):
        X_, z_ = resample(X_train, z_train)
        z_pred[:, j] = model.fit(X_, z_).predict(X_test)

    error[i] = np.mean( np.mean((z_test[:, np.newaxis] - z_pred)**2, axis=1, keepdims=True) )
    bias[i] = np.mean( (z_test - np.mean(z_pred, axis=1, keepdims=True))**2 )
    variance[i] = np.mean( np.var(z_pred, axis=1, keepdims=True) )

plt.plot(polynomialvalues, error, label='Error')
plt.plot(polynomialvalues, bias, label='bias')
plt.plot(polynomialvalues, variance, label='Variance')
plt.title("Error, bias, and variance versus polynomial degree (Bootstrap)")
plt.xticks(np.arange(0, maxdegree+1, step=1))
plt.xlabel("Polynomial degree")
plt.legend()
plt.show()

In [ ]:
maxdegree = 14
k = 10
polynomialvalues = np.arange(start=1, stop=maxdegree+1, step=1)

# Initialize a KFold instance
kfold = KFold(n_splits = k)

# Perform the cross-validation to estimate MSE
Errors_KFold = np.zeros((maxdegree, k))
Biases_KFold = np.zeros((maxdegree, k))
Variances_KFold = np.zeros((maxdegree, k))

z_copy = np.ravel(z)

for i, p in enumerate(polynomialvalues):
    model = linear_model.LinearRegression(fit_intercept=False)
    X = determine_X(x=np.ravel(x), y=np.ravel(y), poly_deg=p)
    for j, (train_inds, test_inds) in enumerate(kfold.split(X)):
        xtrain = X[train_inds]
        ztrain = z_copy[train_inds]

        xtest = X[test_inds]
        ztest = z_copy[test_inds]
        model.fit(xtrain, ztrain[:, np.newaxis])

        zpred = model.predict(xtest)

        Errors_KFold[i,j] = np.mean((ztest-zpred)**2)
        Biases_KFold[i,j] = np.mean((ztest - np.mean(zpred))**2)
        Variances_KFold[i,j] = np.mean((zpred - np.mean(zpred))**2)


Error = np.mean(Errors_KFold, axis = 1)
Bias = np.mean(Biases_KFold, axis = 1)
Variance = np.mean(Variances_KFold, axis = 1)


## Plot
plt.figure()
plt.plot(np.arange(start=1, stop=p+1, step=1), Error, label = 'Error')
plt.plot(np.arange(start=1, stop=p+1, step=1), Bias, label = 'Bias')
plt.plot(np.arange(start=1, stop=p+1, step=1), Variance, label = 'Variance')
plt.xlabel('Polynomial degree')
plt.xticks(np.arange(0, maxdegree+1, step=1))
plt.title("Error, Bias, and Variance versus polynomial degree (k-fold)")
plt.legend()
plt.show()

In [ ]:
maxdegree = 14

polynomialvalues = np.arange(start=1, stop=maxdegree+1, step=1)

# Initialize a KFold instance
kvalues = np.arange(start=5, stop=11, step=1)

# Perform the cross-validation to estimate MSE
Errors_KFold = np.zeros((maxdegree, k))
Biases_KFold = np.zeros((maxdegree, k))
Variances_KFold = np.zeros((maxdegree, k))
Errors=np.zeros(len(kvalues))
z_copy = np.ravel(z)

for m, k in enumerate(kvalues):
    kfold = KFold(n_splits = k)
    for i, p in enumerate(polynomialvalues):
        model = linear_model.LinearRegression(fit_intercept=False)
        X = determine_X(x=np.ravel(x), y=np.ravel(y), poly_deg=p)
        for j, (train_inds, test_inds) in enumerate(kfold.split(X)):
            xtrain = X[train_inds]
            ztrain = z_copy[train_inds]

            xtest = X[test_inds]
            ztest = z_copy[test_inds]
            model.fit(xtrain, ztrain[:, np.newaxis])

            zpred = model.predict(xtest)

            Errors_KFold[i,j] = np.mean((ztest-zpred)**2)


    Errors[m] = np.max(np.mean(Errors_KFold, axis = 1))

plt.plot(kvalues, Errors)
plt.title("Maximum observed error versus k value")
plt.xlabel('K Values')
plt.show()


In [ ]:
maxdegree = 14

polynomialvalues = np.arange(start=1, stop=maxdegree+1, step=1)

# Initialize a KFold instance
kvalues = 5

lambdas = [10**x for x in range(-6, 2, 1)]

f = plt.subplots(2, round(len(lambdas)/2), figsize=(20,10),sharey=True)

for m ,lmb in enumerate(lambdas):
    for i, p in enumerate(polynomialvalues):
        model = linear_model.Ridge(alpha=lmb)
        X = determine_X(x=np.ravel(x), y=np.ravel(y), poly_deg=p)
        for j, (train_inds, test_inds) in enumerate(kfold.split(X)):
            xtrain = X[train_inds]
            ztrain = z_copy[train_inds]

            xtest = X[test_inds]
            ztest = z_copy[test_inds]
            model.fit(xtrain, ztrain[:, np.newaxis])

            zpred = model.predict(xtest)

            Errors_KFold[i,j] = np.mean((ztest-zpred)**2)
            Biases_KFold[i,j] = np.mean((ztest - np.mean(zpred))**2)
            Variances_KFold[i,j] = np.mean((zpred - np.mean(zpred))**2)

    Error = np.mean(Errors_KFold, axis = 1)
    Bias = np.mean(Biases_KFold, axis = 1)
    Variance = np.mean(Variances_KFold, axis = 1)
    

    ## Plot
    plt.subplot(2, round(len(lambdas)/2), m+1)
    plt.plot(np.arange(start=1, stop=p+1, step=1), Error, label = 'Error')
    plt.plot(np.arange(start=1, stop=p+1, step=1), Bias, label = 'Bias')
    plt.plot(np.arange(start=1, stop=p+1, step=1), Variance, label = 'Variance')
    plt.xlabel('Polynomial degree')
    plt.title("Hyperparameter = " + str(lmb))
    plt.legend()
    

plt.suptitle("Error, bias, and variance versus polynomial degree (Ridge)")
plt.show()

In [ ]:
maxdegree = 14

polynomialvalues = np.arange(start=1, stop=maxdegree+1, step=1)

# Initialize a KFold instance
kvalues = 5

lambdas = [10**x for x in range(-6, 2, 1)]

f = plt.subplots(2, round(len(lambdas)/2), figsize=(20,10),sharey=True)

for m ,lmb in enumerate(lambdas):
    for i, p in enumerate(polynomialvalues):
        model = linear_model.Lasso(alpha=lmb)
        X = determine_X(x=np.ravel(x), y=np.ravel(y), poly_deg=p)
        for j, (train_inds, test_inds) in enumerate(kfold.split(X)):
            xtrain = X[train_inds]
            ztrain = z_copy[train_inds]

            xtest = X[test_inds]
            ztest = z_copy[test_inds]
            model.fit(xtrain, ztrain[:, np.newaxis])

            zpred = model.predict(xtest)

            Errors_KFold[i,j] = np.mean((ztest-zpred)**2)
            Biases_KFold[i,j] = np.mean((ztest - np.mean(zpred))**2)
            Variances_KFold[i,j] = np.mean((zpred - np.mean(zpred))**2)

    Error = np.mean(Errors_KFold, axis = 1)
    Bias = np.mean(Biases_KFold, axis = 1)
    Variance = np.mean(Variances_KFold, axis = 1)
    

    ## Plot
    plt.subplot(2, round(len(lambdas)/2), m+1)
    plt.plot(np.arange(start=1, stop=p+1, step=1), Error, label = 'Error')
    plt.plot(np.arange(start=1, stop=p+1, step=1), Bias, label = 'Bias')
    plt.plot(np.arange(start=1, stop=p+1, step=1), Variance, label = 'Variance')
    plt.xlabel('Polynomial degree')
    plt.title("Hyperparameter = " + str(lmb))
    plt.legend()
    

plt.suptitle("Error, bias, and variance versus polynomial degree (Lasso)")
plt.show()

## Part G

In [20]:
import xarray as xr

ds = xr.open_dataset('SRTM_data_Norway_1.tif')


In [ ]:
ds['band_data'].plot()

In [ ]:
# Load the terrain
terrain1 = imread('SRTM_data_Norway_1.tif')
# Show the terrain
plt.figure()
plt.title('Terrain over Norway 1')
plt.imshow(terrain1, cmap='gray', origin='lower')
plt.colorbar()
plt.xlabel('X')
plt.ylabel('Y')
plt.tight_layout()
plt.savefig('Terrain over Norway.png', dpi = 600, bbox_inches='tight')
plt.show()

x = np.arange(0, terrain1.shape[0], step = 1)
y = np.arange(0, terrain1.shape[1], step = 1)
x, y = np.meshgrid(x, y)
z = terrain1    

In [ ]:
# Assuming determine_X, z, x, and y are defined
z_copy = np.ravel(z)


# Custom scorer to handle polynomial degree
def custom_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return mean_squared_error(y, y_pred)

# Initialize the Ridge regression model
linear = linear_model.LinearRegression(fit_intercept=False)

# Initialize the KFold cross-validator
kfold = KFold(n_splits=10)
z_copy = np.ravel(z)
# Perform grid search
best_params = []
for degree in tqdm(polynomialvalues, ncols=50, total=len(polynomialvalues)):
    X = determine_X(x=np.ravel(x), y=np.ravel(y), poly_deg=degree, intercept=False)
    X_train, X_test, z_train, z_test = prep_train_test(X=X, z=z_copy, train_size=0.66)
    X_train, X_test = scaling_mean(X_train, X_test)
    grid_search = GridSearchCV(estimator=linear, param_grid={}, scoring=custom_scorer, cv=kfold, n_jobs=-1)
    grid_search.fit(X_train, z_train)

    best_params.append((degree, grid_search.score(X_test, z_test)))

# Extract the best parameters
best_params_linear = np.array(best_params)
best_degrees_linear = best_params_linear[:, 0]
best_scores_linear = best_params_linear[:, 1]

# Plotting the results

plt.figure(figsize=(10, 6))
plt.plot(best_degrees_linear, best_scores_linear, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('Mean Squared Error')
plt.title('Best Mean Squared Error for each Polynomial Degree')
plt.grid(True)
plt.show()

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
z_copy = np.ravel(z)

# Precompute polynomial features for all degrees
maxdegree = 15
polynomialvalues = np.arange(start=1, stop=maxdegree+1, step=1)
X_poly = [determine_X(x=np.ravel(x), y=np.ravel(y), poly_deg=p, intercept=False) for p in polynomialvalues]

# Define the parameter grid
param_grid = {
    'alpha': [10**x for x in range(-6, 2, 1)],
    'degree': polynomialvalues
}

# Custom scorer to handle polynomial degree
def custom_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return mean_squared_error(y, y_pred)

# Initialize the Ridge regression model
ridge = linear_model.Ridge(fit_intercept=False)

# Initialize the KFold cross-validator
kfold = KFold(n_splits=10)
z_copy = np.ravel(z)
# Perform grid search
best_params = []
for degree in tqdm(polynomialvalues, ncols=50, total=len(polynomialvalues)):
    X = determine_X(x=np.ravel(x), y=np.ravel(y), poly_deg=degree, intercept=False)
    X_train, X_test, z_train, z_test = prep_train_test(X=X, z=z_copy, train_size=0.66)
    X_train, X_test = scaling_mean(X_train, X_test)
    grid_search = GridSearchCV(estimator=ridge, param_grid={'alpha': param_grid['alpha']}, scoring=custom_scorer, cv=kfold, n_jobs=-1)
    grid_search.fit(X_train, z_train)

    best_params.append((degree, grid_search.best_params_['alpha'], grid_search.score(X_test, z_test)))

# Extract the best parameters
best_params_ridge = np.array(best_params)
best_degrees_ridge = best_params_ridge[:, 0]
best_alphas_ridge = best_params_ridge[:, 1]
best_scores_ridge = best_params_ridge[:, 2]

# Plotting the results

plt.figure(figsize=(10, 6))
plt.plot(best_degrees_ridge, best_scores_ridge, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('Mean Squared Error')
plt.title('Best Mean Squared Error for each Polynomial Degree')
plt.grid(True)
plt.show()

In [ ]:
z_copy = np.ravel(z)

# Define the parameter grid
param_grid = {
    'alpha': [10**x for x in range(-6, 1, 1)],
    'degree': polynomialvalues
}

# Custom scorer to handle polynomial degree
def custom_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return mean_squared_error(y, y_pred)

# Initialize the Ridge regression model
lasso = linear_model.Lasso(fit_intercept=False, tol=0.01)

# Initialize the KFold cross-validator
kfold = KFold(n_splits=10)
z_copy = np.ravel(z)
# Perform grid search
best_params = []
scaler = StandardScaler(with_std=False)
for degree in tqdm(polynomialvalues, ncols=50, total=len(polynomialvalues)):
    X = determine_X(x=np.ravel(x), y=np.ravel(y), poly_deg=degree, intercept=False)
    X_train, X_test, z_train, z_test = prep_train_test(X=X, z=z_copy, train_size=0.66)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    grid_search = GridSearchCV(estimator=lasso, param_grid={'alpha': param_grid['alpha']}, scoring=custom_scorer, cv=kfold, n_jobs=-1)
    grid_search.fit(X_train, z_train)

    best_params.append((degree, grid_search.best_params_['alpha'], grid_search.score(X_test, z_test)))

# Extract the best parameters
best_params_lasso = np.array(best_params)
best_degrees_lasso = best_params_lasso[:, 0]
best_alphas_lasso = best_params_lasso[:, 1]
best_scores_lasso = best_params_lasso[:, 2]

# Plotting the results

plt.figure(figsize=(10, 6))
plt.plot(best_degrees_lasso, best_scores_lasso, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('Mean Squared Error')
plt.title('Best Mean Squared Error for each Polynomial Degree')
plt.grid(True)
plt.show()

In [ ]:
print('MSE of OLS:', np.min(best_scores_linear))
print('Best polynomial degree for OLS:', best_degrees_linear[np.argmin(best_scores_linear)])

print('MSE of Ridge:', np.min(best_scores_ridge))
print('Best polynomial degree for Ridge:', best_degrees_ridge[np.argmin(best_scores_ridge)])
print('Best alpha for Ridge:', best_alphas_ridge[np.argmin(best_scores_ridge)])

print('MSE of Lasso:', np.min(best_scores_lasso))
print('Best polynomial degree for Lasso:', best_degrees_lasso[np.argmin(best_scores_lasso)])
print('Best alpha for Lasso:', best_alphas_lasso[np.argmin(best_scores_lasso)])

In [ ]:
#Hyper parameters for ridge and lasso:
print('Hyperparameters for Linear:', pd.DataFrame(best_params_linear, columns=['Degree', 'MSE']))
print('Hyperparameters for Ridge:', pd.DataFrame(best_params_ridge, columns=['Degree', 'Alpha', 'MSE']))
print('Hyperparameters for Lasso:', pd.DataFrame(best_params_lasso, columns=['Degree', 'Alpha', 'MSE']))